In [0]:
import re
import os

import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, csc_matrix, eye


# Exercise 3: Sparse Matrices, Static and Transient Problem of Governor

a) The mass and stiffness matrices were exported from Ansys in the Harwell-Boeing format for sparse matrices. Write a function to import the matrices located in the files M_L.txt and K_L.txt.

In [2]:
def conv(value):
    try:
        return float(value)
    except ValueError:
        return float(re.sub(r"D", 'E', str(value, 'utf-8')))

def read_hb(file_path):
    with open(file_path) as file:
        line = file.readline().strip("\n")
        if not len(line) > 72:
            raise ValueError("Expected at least 72 characters for first line, "
                             "got: \n%s" % line)
        title = line[:72]
        key = line[72:]

        # Second line
        line = file.readline().strip("\n")
        if not len(line.rstrip()) >= 56:
            raise ValueError("Expected at least 56 characters for second line, "
                         "got: \n%s" % line)
        total_nlines = int(line[:14])
        pointer_nlines = int(line[14:28])
        indices_nlines = int(line[28:42])
        values_nlines = int(line[42:56])

        data = np.loadtxt(file_path, skiprows=5, converters={0: conv})

        indptr = data[:pointer_nlines].astype(int) - 1
        indices = data[pointer_nlines:pointer_nlines + indices_nlines].astype(int) - 1
        values = data[pointer_nlines + indices_nlines:pointer_nlines + indices_nlines + values_nlines]

    return csc_matrix((values, indices, indptr))

ValueError: Only files without right hand side supported for now.

Execute function ```read_hb```.

In [ ]:
resources_path = os.path.join('resources')
K_L_path = os.path.join(resources_path, 'K_L.txt')
M_L_path = os.path.join(resources_path, 'M_L.txt')

K_txt = read_hb(K_L_path)
M_txt = read_hb(M_L_path)

Compare the results with the already imported matrices

In [ ]:
from pymatreader import read_mat

def read_hb_from_mat(file_path, name):
    mat_dic = read_mat(file_path)[name]
    indptr = mat_dic['jc']
    indices = mat_dic['ir']
    values = mat_dic['data']
    return csc_matrix((values, indices, indptr))

In [ ]:
K_mat_path = os.path.join(resources_path, 'K.mat')
M_mat_path = os.path.join(resources_path, 'M.mat')

K_mat = read_hb_from_mat(K_mat_path, 'K')
M_mat = read_hb_from_mat(M_mat_path, 'M')

#TODO equality check

Alternative use already implemented functions
```
scipy.io.hb_read
scipy.io.hb_write
```

In [ ]:
from scipy.io.harwell_boeing import hb_read, hb_write
K_hb_path = os.path.join(resources_path, 'K.hb')
M_hb_path = os.path.join(resources_path, 'M.hb')
hb_write(M_hb_path, M_mat)
hb_write(K_hb_path, K_mat)

M_hb = hb_read(M_hb_path)
K_hb = hb_read(K_hb_path)